In [25]:
from database import SessionLocal, engine
import crud, models
import pandas as pd

In [28]:
transaction2022 = pd.read_csv('preparation/transaction_data_2022.csv')
transaction2022['transaction_date'] = pd.to_datetime(transaction2022['transaction_date'])
transaction2022.head()

,transaction_date,amount,wallet_id,category_id,description,transaction_type_id
0,2022-07-02,290000,4,18.0,NaN,1
1,2022-07-02,464300,1,9.0,NaN,1
2,2022-07-02,15000,4,7.0,NaN,1
3,2022-07-03,65000,4,9.0,NaN,1
4,2022-07-03,70000,4,7.0,NaN,1


In [30]:
models.Base.metadata.create_all(bind=engine)
def get_db():
    db = SessionLocal()

    return db
db = get_db()
user_id = 1

for _, row in transaction2022.iterrows():
    crud.create_transaction(db,
                        user_id,
                        wallet_id=row['wallet_id'],
                        category_id=row['category_id'],
                        transaction_type_id=row['transaction_type_id'],
                        amount=row['amount'],
                        transaction_date=row['transaction_date'])

In [24]:
transactions = crud.get_transactions(db, user_id=user_id)
transactions_df = pd.DataFrame([transaction.__dict__ for transaction in transactions])

assets_wallets = crud.get_wallets(db, user_id=user_id, liability=0)
assets_transactions_df = transactions_df[transactions_df['wallet_id'].isin([wallet.id for wallet in assets_wallets])]
positve_df = assets_transactions_df[assets_transactions_df['transaction_type_id'] != 1]
expense_df = assets_transactions_df[assets_transactions_df['transaction_type_id'] == 1]

,wallet_id,positive,negative,balance
0,1,343429861.0,284067420.0,59362441.0
1,2,11001866.0,11723434.0,-721568.0
2,3,6122403.0,84126355.0,-78003952.0
3,4,89557480.0,0.0,89557480.0
4,5,176161158.0,19057175.0,157103983.0
